In [1]:
import os
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import prepare_calculations
import explore_qml_data as eqd
import utils_qm as uqm

sys.path.insert(0, '/home/misa/git_repositories/xyz2mol_modified/')
import xyz2mol

# Amons - Generate Input

In [2]:
amons_dict = uqm.load_obj('/home/misa/datasets/amons_qm9_11k/unique_amons_dict')
smiles_upto_30ve = []
with open('/home/misa/projects/Atomic-Energies/data/ueg_reference/amons/batch1_up_to_30_ve.txt') as f:
    for line in f:
        smiles_upto_30ve.append(line.rstrip('\n'))

In [7]:
#smiles = smiles_upto_30ve[1]

for i, smiles in enumerate(smiles_upto_30ve):
    if smiles == 'C':
        print(i+1)
        
    xyz_data = amons_dict[smiles]
    
    num_digits_i = len(str(i+1))
    leading_zeros = (6 - num_digits_i)*'0'
    compound_name = f'amon_{leading_zeros}{i+1}'
    compound_path = f'/home/misa/projects/Atomic-Energies/data/ueg_reference/amons/{compound_name}'

    lambda_values = np.array([0.2, 0.4, 0.6, 0.8, 1.0])
    nuc_charges, charge, coords_initial = xyz2mol.read_xyz_data(xyz_data)
    atom_symbols = []
    for n in nuc_charges:
        atom_symbols.append(xyz2mol.str_atom(n).upper())

    # calculation parameters (independent of lambda value)
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = prepare_calculations.get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = prepare_calculations.get_gpts(num_ve) # get gridpoints
    num_gpts = num_gpts_higher

    # shift molecule to center of box
    coords = eqd.shift2center(coords_initial, np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val in lambda_values:
        new_lambda, scaled_ve = prepare_calculations.get_lambda(lam_val, num_ve)
        # scaled_ve is number of electrons added from pseudopotential file, the remaining electrons must be added in form of a negative charge
        charge = scaled_ve - num_ve # write input

        # create directory if necessary
        if scaled_ve < 10:
            scaled_ve_str = '0'+str(scaled_ve)
        else:
            scaled_ve_str = str(scaled_ve)
        work_dir = os.path.join(compound_path, f've_{scaled_ve_str}/')
        os.makedirs(work_dir, exist_ok=True)

        # generate input file
        input_path = os.path.join(work_dir, 'run.inp')
        prepare_calculations.write_input(atom_symbols, charge, coords, num_gpts, boxsize, input_path, '/home/misa/projects/Atomic-Energies/data/cpmd_params_template_pbe.inp', debug = False)

        # generate pp-files
        prepare_calculations.write_pp_files_compound(atom_symbols, new_lambda, work_dir, pp_dir='/home/misa/software/Goedecker/library/pbe', pp_type='_GH_PBE')
        
        # write smiles
        with open(f'/home/misa/projects/Atomic-Energies/data/ueg_reference/amons/{compound_name}/smiles', 'w') as f:
            f.write(smiles+'\n')

1


In [8]:
len(smiles_upto_30ve)

801

# QM9

In [ ]:
comp = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_000930.xyz')
charge = 0
box = 20.0
gpts = 350
els = comp.atomtypes
coords = comp.coordinates
write_path = '/home/misa/projects/Atomic-Energies/data/test.inp'
prepare_calculations.write_input(els, charge, coords, gpts, box, write_path, template_path='/home/misa/projects/Atomic-Energies/data/cpmd_params_template.inp', debug = False)

In [ ]:
selected_molecules = [
 '/home/misa/datasets/qm9/dsgdb9nsd_000940.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000949.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_002102.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_002110.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000001.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000007.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000013.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000039.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000133.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000543.xyz']

In [ ]:
for s in selected_molecules:
    s.split('.')[0].split('/')[-1]

In [ ]:
for s in selected_molecules:
    path_to_xyz = s#'/home/misa/datasets/qm9/dsgdb9nsd_000930.xyz'
    compound = s.split('.')[0].split('/')[-1]
    compound_path = f'/home/misa/projects/Atomic-Energies/data/ueg_reference/SCF_calculations/{compound}/'
    lambda_values = np.array([0.2, 0.4, 0.6, 0.8, 1.0])

    comp = qml.Compound(xyz=path_to_xyz)
    atomsymbols = comp.atomtypes
    nuc_charges = comp.nuclear_charges

    # calculation parameters (independent of lambda value)
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = prepare_calculations.get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = prepare_calculations.get_gpts(num_ve) # get gridpoints
    num_gpts = num_gpts_higher

    # shift molecule to center of the box
    coords_initial = comp.coordinates
    coords = eqd.shift2center(coords_initial, np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val in lambda_values:
        new_lambda, scaled_ve = prepare_calculations.get_lambda(lam_val, num_ve)
        # scaled_ve is number of electrons added from pseudopotential file, the remaining electrons must be added in form of a negative charge
        charge = scaled_ve - num_ve # write input

        # create directory if necessary
        if scaled_ve < 10:
            scaled_ve_str = '0'+str(scaled_ve)
        else:
            scaled_ve_str = str(scaled_ve)
        work_dir = os.path.join(compound_path, f've_{scaled_ve_str}/')
        os.makedirs(work_dir, exist_ok=True)

        # generate input file
        input_path = os.path.join(work_dir, 'run.inp')
        prepare_calculations.write_input(atomsymbols, charge, coords, num_gpts, boxsize, input_path, '/home/misa/projects/Atomic-Energies/data/cpmd_params_template.inp', debug = False)

        # generate pp-files
        prepare_calculations.write_pp_files_compound(comp, new_lambda, work_dir, pp_dir='/home/misa/software/PP_LIBRARY/', pp_type='_SG_LDA')

In [ ]:
comp.natypes.keys()

# Calculation of Density Slices

In [ ]:
import os
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import prepare_calculations
import explore_qml_data as eqd

import ase

In [ ]:


# get position of grid points

# move to closest grid point

In [ ]:
compound = 'dsgdb9nsd_000930'
compound_path = f'/home/misa/projects/Atomic-Energies/data/ueg_reference/effect_heteroatoms/{compound}/'
lambda_values = np.array([0.2, 0.4, 0.6, 0.8, 1.0])

atoms = ase.io.read('/home/misa/projects/Atomic-Energies/data/ueg_reference/effect_heteroatoms/dsgdb9nsd_000930/dsgdb9nsd_000930_atoms.xyz')
#shift1 = atoms.get_positions()+np.array([24,4,20])
#print(shift1)

atomsymbols = atoms.get_chemical_symbols()
nuc_charges = atoms.get_atomic_numbers()

# calculation parameters (independent of lambda value)
num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
boxsize = prepare_calculations.get_boxsize(num_ve) # get boxsize
num_gpts_lower, num_gpts_higher = prepare_calculations.get_gpts(num_ve) # get gridpoints
num_gpts = num_gpts_higher

# shift molecule to center of box
coords_initial = atoms.get_positions()
coords = eqd.shift2center(coords_initial, np.array([boxsize, boxsize, boxsize])/2)
# shift molecule such that atom closest to heteroatom is at gpt
index = 1
pos_gpts = np.arange(num_gpts)*boxsize/num_gpts
shift_z = pos_gpts[np.where(np.abs(coords[1,2]-pos_gpts) == np.amin(np.abs(coords[1,2]-pos_gpts)))] - coords[index, 2]
shift_vector = np.array([0, 0, shift_z[0]])
shifted_coords = coords+shift_vector


# get correct lambda value
for lam_val in lambda_values:
    new_lambda, scaled_ve = prepare_calculations.get_lambda(lam_val, num_ve)
    # scaled_ve is number of electrons added from pseudopotential file, the remaining electrons must be added in form of a negative charge
    charge = scaled_ve - num_ve # write input

    # create directory if necessary
    if scaled_ve < 10:
        scaled_ve_str = '0'+str(scaled_ve)
    else:
        scaled_ve_str = str(scaled_ve)
    work_dir = os.path.join(compound_path, f've_{scaled_ve_str}/')
    os.makedirs(work_dir, exist_ok=True)

    # generate input file
    input_path = os.path.join(work_dir, 'run.inp')
    prepare_calculations.write_input(atomsymbols, charge, coords, num_gpts, boxsize, input_path, '/home/misa/projects/Atomic-Energies/data/cpmd_params_template.inp', debug = False)

    # generate pp-files
    #prepare_calculations.write_pp_files_compound(comp, new_lambda, work_dir, pp_dir='/home/misa/software/PP_LIBRARY/', pp_type='_SG_LDA')

In [ ]:
shifted_coords

In [ ]:
np.amin(np.abs(coords[1,2]+shift_vector[2]-pos_gpts))

In [ ]:
(coords+shift_vector)[:,2]